# Calibration for existing modes (Seaport trips only)
1. Read current mode share and param file
2. Create new mode and new param files for calibration

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import openpyxl
import os
from shutil import copyfile
import glob
import sys
import importlib

In [2]:
os.getcwd()

'C:\\Users\\xchang\\OneDrive - Cambridge Systematics\\Documents\\GitHub\\cs_fm_tool\\calibration'

In [3]:
os.chdir('..')

In [4]:
import cs_fm_tool
import config
import mode_choice.mc_util as util

In [4]:
fm = cs_fm_tool.CS_FM_Tool(config, 'projects/baseline_seaport_2018.yaml','param/perf_meas_nosm.yaml')
fm.load_inputs()
fm.setup()
fm.run()
util.write_study_area_mode_share_to_excel(fm.mc, r"C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\model_data\calibration\base_share_for_calib.xlsx")

✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.09 seconds
✓ skims read. Time elapsed: 8.46 seconds
✓ trip table read. Time elapsed: 11.51 seconds
✓ zonal variable tables generated. Time elapsed: 11.79 seconds
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 11.86 seconds
✓ Trips for 0_PK calculated. Time elapsed: 16.29 seconds
✓ Trips for 1_PK calculated. Time elapsed: 20.71 seconds
✓ Trips for 0_OP calculated. Time elapsed: 25.53 seconds
✓ Trips for 1_OP calculated. Time elapsed: 30.38 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 30.58 seconds
✓ Trips for 0_PK calculated. Time elapsed: 33.94 seconds
✓ Trips for 1_PK calculated. Time elapsed: 37.54 seconds
✓ Trips for 0_OP calculated. Time elapsed: 41.25 seconds
✓ Trips for 1_OP calculated. Time elapsed: 44.77 seconds
Mode choice for NHB started.
✓ Parameter table for NHB generated. Time elapsed: 44.93 seconds
✓ Tri

In [5]:
purposes =['HBW','HBO','NHB', 'HBSc1','HBSc2','HBSc3']
peak_veh = ['0_PK','1_PK','0_OP','1_OP']

In [6]:
def update_ASC(mc_summary, old_param, new_param):
    book = openpyxl.Workbook()
    book.save(new_param)

    for purpose in purposes:
        model = pd.read_excel(mc_summary,sheet_name = purpose,index_col = 0)

        modes = model.index
        df = pd.DataFrame(index = modes, columns = peak_veh)
        for pv in peak_veh:
            # compare if shares are higher or lower than what's supposed to happen
            _t = target[(target['Purpose'] == purpose) & (target['Segment'] == pv)]
            for mode in modes:
                target_share = _t[mode].values
                model_value = model.loc[mode][pv] # convert to share
                model_share = model_value / model[pv].sum()
                
                if target_share != 0 and abs((model_share - target_share)/target_share)<0.05:
                    df.loc[mode,pv] = 0
                elif target_share == 0 and abs(model_share - target_share)<0.05:
                    df.loc[mode,pv] = 0
                elif model_share != 0 and target_share == 0:
                    df.loc[mode,pv] = -1
                else:
                    df.loc[mode,pv] = np.log(target_share / model_share)[0]
        
        # read param, output new ASCs
        ASC = pd.read_excel(old_param, sheet_name = purpose,index_col = 0).filter(regex='ASC')
        param = pd.read_excel(old_param, sheet_name = purpose,index_col = 0).drop(ASC.columns,axis = 1)

        ASC.rename(columns = dict(zip(['ASC_'+pv for pv in peak_veh],peak_veh)),inplace = True)
        ASC_new = ASC + df

        # find 0-ASC mode
        for pv in peak_veh:
            ASC_0 = ASC.index[ASC[pv] == 0].values[0]
            ASC_new[pv] -= ASC_new.loc[ASC_0,pv]

        # make sure ASCs have absolute value <= 6    
        ASC_new[ASC_new>6] = 6
        ASC_new[ASC_new<-6] = -6
        
        # write to a new param file
        book = openpyxl.load_workbook(new_param)
        writer = pd.ExcelWriter(new_param,engine = 'openpyxl')
        writer.book = book

        param_new = pd.concat([param,
        ASC_new.rename(columns = dict(zip(peak_veh,['ASC_'+pv for pv in peak_veh])))],axis = 1)
        param_new.reset_index(inplace = True)
        param_new.to_excel(writer,sheet_name = purpose,index = False)
        writer.save()

In [9]:
def new_run(iter_dir,param_new):
    try: os.mkdir(iter_dir)
    except: print('Directory exists.')   
        
    fm = cs_fm_tool.CS_FM_Tool(config, 'projects/baseline_seaport_2018.yaml','param/perf_meas_nosm.yaml')
    fm.mc.param_file = param_new
    fm.load_inputs()
    fm.setup()
    fm.run()

    util.write_study_area_mode_share_to_excel(fm.mc,out_excel_fn = iter_dir + 'MC_mode_share.xlsx')

# Calibration loop for regular modes (without SM)

In [10]:

target = pd.read_csv(r"C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\model_data\calibration\MC_TargetShares_SPRT_2018.csv")
calibration_run_path = r'C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\model_data\calibration\\'
mc_summary_file = calibration_run_path + "base_share_for_calib.xlsx"
param_file = r"C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\model_data\Seaport_data\param\param_calib_0716.xlsx"
param_new = calibration_run_path + "param_iter1.xlsx"

update_ASC(mc_summary_file, param_file, param_new)
new_run(calibration_run_path + r"iter_1\\",param_new)

for i in range(2,10):
    update_ASC(calibration_run_path+ "iter_{}\MC_mode_share.xlsx".format(i-1), 
               calibration_run_path+ "param_iter{}.xlsx".format(i-1),
               calibration_run_path + "param_iter{}.xlsx".format(i))
    
    iter_dir = calibration_run_path+r"iter_{}\\".format(i)
    new_run(iter_dir, calibration_run_path + "param_iter{}.xlsx".format(i))

✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.05 seconds
✓ skims read. Time elapsed: 7.67 seconds
✓ trip table read. Time elapsed: 10.32 seconds
✓ zonal variable tables generated. Time elapsed: 10.49 seconds
Running for all purposes...
Mode choice for HBW started.
C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\model_data\calibration\\param_iter1.xlsx
✓ Parameter table for HBW generated. Time elapsed: 10.51 seconds
✓ Trips for 0_PK calculated. Time elapsed: 14.89 seconds
✓ Trips for 1_PK calculated. Time elapsed: 18.98 seconds
✓ Trips for 0_OP calculated. Time elapsed: 23.22 seconds
✓ Trips for 1_OP calculated. Time elapsed: 27.25 seconds
Mode choice for HBO started.
C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\model_data\calibration\\param_iter1.xlsx
✓ Parameter table for HBO generated. Time elapsed: 27.47 seconds
✓ Trips for 0_PK calculated. Time elapsed: 30.29 seconds
✓ Trips for 1_PK calculated. Time elapsed: 33.09 seconds

# Check post-calibration shares

In [6]:
fm = cs_fm_tool.CS_FM_Tool(config, 'projects/calibration_seaport_2018.yaml','param/perf_meas_nosm.yaml')
fm.load_inputs()
fm.setup()
fm.run()


✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.07 seconds
✓ skims read. Time elapsed: 7.76 seconds
✓ trip table read. Time elapsed: 10.47 seconds
✓ zonal variable tables generated. Time elapsed: 10.62 seconds
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 10.65 seconds
✓ Trips for 0_PK calculated. Time elapsed: 14.94 seconds
✓ Trips for 1_PK calculated. Time elapsed: 19.10 seconds
✓ Trips for 0_OP calculated. Time elapsed: 23.44 seconds
✓ Trips for 1_OP calculated. Time elapsed: 27.82 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 28.05 seconds
✓ Trips for 0_PK calculated. Time elapsed: 31.15 seconds
✓ Trips for 1_PK calculated. Time elapsed: 34.39 seconds
✓ Trips for 0_OP calculated. Time elapsed: 37.67 seconds
✓ Trips for 1_OP calculated. Time elapsed: 41.10 seconds
Mode choice for NHB started.
✓ Parameter table for NHB generated. Time elapsed: 41.25 seconds
✓ Tri

In [7]:
util.mode_share_by_subarea(fm.mc, r"C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\model_data\calibration\mode_share_iter9.csv")

# Smart Mobility (to be developed)

In [ ]:
# change these paths
mc_script_source_path = r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Scenarios\2040\\"
sm_calibration_run_path = r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Utilities\smart mobility\smart mobility\\"
try:  os.mkdir(sm_calibration_run_path + 'SM calibration')
except: print('Directory exists.')

sys.path.append(mc_script_source_path)

In [22]:
import mode_choice; import mc_util; import config
mc = mode_choice.Mode_Choice(config,run_now = True, all_purposes = True)

In [4]:
# pick up results of the baseline run with no SM mode in parameter
current_share_file = r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Scenarios\Test Output\MC_mode_share_20180801_163322.xlsx"
current_param_file = config.param_file

In [5]:
# Read baseline mode share output and desired SM shares, create new mode share target by redistributing mode shares using MAPC numbers
SM_target = pd.read_csv(sm_calibration_run_path+"SM_target_share.csv")
SM_target['SM_RA'] = SM_target['SM_RA'].str.rstrip('%').astype('float') / 100.0
SM_target['SM_SH'] = SM_target['SM_SH'].str.rstrip('%').astype('float') / 100.0

# SM_target file currently specifies: HBO: 5% SM_RA, 5% SM_SH
# Other purposes: 0.5% SM_RA, 0.5% SM_SH

# parse current mode share into share target format

purposes =['HBW','HBO','NHB', 'HBSc1','HBSc2','HBSc3']
peak_veh = ['0_PK','1_PK','0_OP','1_OP']
target_df = pd.DataFrame(columns = ['Purpose','PV','DA','SR2','SR3+','Bike','Walk','WAT','DAT_B','DAT_CR','DAT_RT','DAT_LB'])
for purpose in purposes:
    shares = pd.read_excel(current_share_file, sheet_name = purpose).T.loc[peak_veh].reset_index().rename(columns = {'index':'PV'})
    shares['Purpose'] = purpose
    # normalize
    mode_cols = shares.columns.drop(['PV','Purpose'])
    shares[mode_cols] = shares[mode_cols].divide(shares[mode_cols].sum(axis = 1), axis = 0)
    target_df = target_df.append(shares,)
target_df.fillna(0,inplace = True)


In [13]:
# redistribute mode share based on MAPC survey
walk_bike_factor = 112./669 # proportional split, walk and bike
drive_factor = 167./669 # DA, SR2, SR3+
transit_factor = 390./669 # WAT and DAT

def mode_share_adj(row):
    SM_total = row[['SM_RA','SM_SH']].sum()
    new_vals = dict.fromkeys(['Walk','Bike','DA','SR2','SR3+','WAT','DAT_B','DAT_CR','DAT_LB','DAT_RT'])
    new_vals['Walk'] = max(row.Walk - SM_total * walk_bike_factor * row.Walk / (row.Walk + row.Bike),0.5*row.Walk)
    new_vals['Bike'] = max(row.Bike - SM_total * walk_bike_factor * row.Bike / (row.Walk + row.Bike),0.5*row.Bike)
    new_vals['DA'] = max(row.DA - SM_total * drive_factor * row.DA / (row.DA + row.SR2 + row['SR3+']),0.5*row.DA)
    new_vals['SR2'] = max(row.SR2 - SM_total * drive_factor * row.SR2 / (row.DA + row.SR2 + row['SR3+']),0.5*row.SR2)
    new_vals['SR3+'] = max(row['SR3+'] - SM_total * drive_factor * row['SR3+'] / (row.DA + row.SR2 + row['SR3+']) , 0.5*row['SR3+'])
    new_vals['WAT'] = max(row.WAT - SM_total * transit_factor * row.WAT / (row.WAT + row.DAT_B + row.DAT_CR + row.DAT_LB + row.DAT_RT),0.5*row.WAT)
    new_vals['DAT_B'] = max(row.DAT_B - SM_total * transit_factor * row.DAT_B / (row.WAT + row.DAT_B + row.DAT_CR + row.DAT_LB + row.DAT_RT),0.5*row.DAT_B)
    new_vals['DAT_CR'] = max(row.DAT_CR - SM_total * transit_factor * row.DAT_CR / (row.WAT + row.DAT_B + row.DAT_CR + row.DAT_LB + row.DAT_RT),0.5*row.DAT_CR)
    new_vals['DAT_LB'] = max(row.DAT_LB - SM_total * transit_factor * row.DAT_LB / (row.WAT + row.DAT_B + row.DAT_CR + row.DAT_LB + row.DAT_RT),0.5*row.DAT_LB)
    new_vals['DAT_RT'] = max(row.DAT_RT - SM_total * transit_factor * row.DAT_RT / (row.WAT + row.DAT_B + row.DAT_CR + row.DAT_LB + row.DAT_RT),0.5*row.DAT_RT)
    
    #  if the share reduction is not enough, take shares from the top three
    if sum(list(new_vals.values())) + SM_total >1:
        share_to_reduce = (sum(list(new_vals.values())) + SM_total - 1)
        modes_to_reduce = (sorted(new_vals.keys(), key=lambda k: new_vals[k],reverse = True)[:3])
        for mode in modes_to_reduce:
            new_vals[mode]-= share_to_reduce / 3
    return pd.Series(new_vals)
    
    
target = target_df.merge(SM_target, on = ['Purpose','PV'],how ='left')
conv_mode_cols = target.columns.drop(['PV','Purpose','SM_RA','SM_SH'])
target[conv_mode_cols] =target.apply(mode_share_adj,axis = 1)


In [7]:
# add new modes in baseline mode share file
writer = pd.ExcelWriter(sm_calibration_run_path+'base_share_for_calib.xlsx',engine = 'openpyxl')
for purpose in purposes:
    shares = pd.read_excel(current_share_file,sheet_name = purpose)
    shares.loc['SM_RA'] = 0
    shares.loc['SM_SH'] = 0
    shares.to_excel(writer, sheet_name = purpose )
writer.save()

In [8]:
# create new param file that contains ASC for the two new modes
writer = pd.ExcelWriter(sm_calibration_run_path+'param_with_SM.xlsx',engine = 'openpyxl')
for purpose in purposes:
    param = pd.read_excel(current_param_file,sheet_name = purpose,index_col = 0)
    SM_nest = param.nest.max()+1
    SM_RA_nest_coef = 0.7
    SM_SH_nest_coef = 0.7
    if 'DA' in param.index:
        IVTT = param.loc['DA','IVTT']
        OVTT = param.loc['DA','OVTT']
        Cost = param.loc['DA','Cost']
    else: 
        IVTT = param.loc['SR2','IVTT']
        OVTT = param.loc['SR2','OVTT']
        Cost = param.loc['SR2','Cost']
    ASC = -6 
    SM_RA = pd.Series({'nest':SM_nest,'nest_coefficient':SM_RA_nest_coef,'IVTT':IVTT,'OVTT':OVTT,'Cost':Cost,
    'ASC_0_PK':ASC, 'ASC_1_PK':ASC,'ASC_0_OP':ASC, 'ASC_1_OP':ASC })
    SM_RA.name = 'SM_RA'

    SM_SH = pd.Series({'nest':SM_nest,'nest_coefficient':SM_SH_nest_coef,'IVTT':IVTT,'OVTT':OVTT,'Cost':Cost,
    'ASC_0_PK':ASC, 'ASC_1_PK':ASC,'ASC_0_OP':ASC, 'ASC_1_OP':ASC })
    SM_SH.name = 'SM_SH'    
    
    param = param.append([SM_RA,SM_SH])
    param.index.name = 'mode'
    param.to_excel(writer, sheet_name = purpose)

writer.save()

C:\Users\xchang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:5186: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  other = other.loc[:, self.columns]
C:\Users\xchang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [9]:
def update_ASC(mc_summary, old_param,new_param):
    book = openpyxl.Workbook()
    book.save(new_param)

    for purpose in purposes:
        model = pd.read_excel(mc_summary,sheet_name = purpose,index_col = 0)

        modes = model.index
        df = pd.DataFrame(index = modes, columns = peak_veh)
        for pv in peak_veh:
            # compare if shares are higher or lower than what's supposed to happen
            _t = target[(target['Purpose'] == purpose) & (target['PV'] == pv)]
            for mode in modes:
                target_share = _t[mode].values
                model_value = model.loc[mode][pv] # convert to share
                model_share = model_value / model[pv].sum()
                
                if target_share != 0 and abs((model_share - target_share)/target_share)<0.05:
                    df.loc[mode,pv] = 0
                elif target_share == 0 and abs(model_share - target_share)<0.05:
                    df.loc[mode,pv] = 0
                elif model_share != 0 and target_share == 0:
                    df.loc[mode,pv] = -1
                else:
                    df.loc[mode,pv] = np.log(target_share / model_share)[0]
        
        
        # read param, output new ASCs
        ASC = pd.read_excel(old_param, sheet_name = purpose,index_col = 0).filter(regex='ASC')
        param = pd.read_excel(old_param, sheet_name = purpose,index_col = 0).drop(ASC.columns,axis = 1)

        ASC.rename(columns = dict(zip(['ASC_'+pv for pv in peak_veh],peak_veh)),inplace = True)
        ASC_new = ASC + df

        # find 0-ASC mode
        for pv in peak_veh:
            ASC_0 = ASC.index[ASC[pv] == 0].values[0]
            ASC_new[pv] -= ASC_new.loc[ASC_0,pv]

        # make sure ASCs have absolute value <= 6    
        ASC_new[ASC_new>6] = 6
        ASC_new[ASC_new<-6] = -6
        
        # write to a new param file
        book = openpyxl.load_workbook(new_param)
        writer = pd.ExcelWriter(new_param,engine = 'openpyxl')
        writer.book = book

        param_new = pd.concat([param,
        ASC_new.rename(columns = dict(zip(peak_veh,['ASC_'+pv for pv in peak_veh])))],axis = 1)
        param_new.reset_index(inplace = True)
        param_new.to_excel(writer,sheet_name = purpose,index = False)
        writer.save()


In [10]:
def new_run(iter_dir,param_new):
    try: os.mkdir(iter_dir)
    except: print('Directory exists.')

    base_config = mc_script_source_path + "config.py"
    new_config = iter_dir+r"\config.py"

    # call mode choice, use new parameter file
    with open(base_config,'r') as cfg:
        txt = cfg.read()
        txt_new = txt.replace(r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Mode Choice\param_calib_0716.xlsx",param_new)
    with open(new_config,'w') as cfg1:
        cfg1.write(txt_new)

    # copy model run files
    for src in glob.iglob(mc_script_source_path + "m*.py"):
        fn = src.split('\\')[-1]
        copyfile(src, r"{}\{}".format(iter_dir,fn))

    # copy calibration notebook
    copyfile(sm_calibration_run_path + "base_share_for_calib.xlsx",iter_dir+r"\MC_mode_share.xlsx")

    # replace output directory in mc_util
    output_org = r'out_path + "MC_mode_share_{}.xlsx"'
    output_new = 'r"' + iter_dir+r'\MC_mode_share.xlsx"'
    with open(iter_dir + r"\mc_util.py",'r') as util:
        txt = util.read()
        txt_new = txt.replace(output_org,output_new)
    with open(iter_dir + r"\mc_util.py",'w') as util1:
        util1.write(txt_new)

    sys.path.insert(0, iter_dir)
    if 'mode_choice' in sys.modules:  
        del sys.modules["mode_choice"]

    if 'mc_util' in sys.modules:  
        del sys.modules["mc_util"]

    if 'config' in sys.modules:
        del sys.modules['config']

    import mode_choice; import mc_util; import config

    mc = mode_choice.Mode_Choice(config,run_now = True, all_purposes = True)

In [15]:
mc_summary_file = sm_calibration_run_path+ "base_share_for_calib.xlsx"
param_file =sm_calibration_run_path+"param_with_SM.xlsx"
param_new = sm_calibration_run_path"param_iter1.xlsx"

if 'mode_choice' in sys.modules:  
    del sys.modules["mode_choice"]

if 'mc_util' in sys.modules:  
    del sys.modules["mc_util"]

if 'config' in sys.modules:
    del sys.modules['config']

import mode_choice; import mc_util; import config

update_ASC(mc_summary_file, param_file, param_new)
new_run(sm_calibration_run_path + "SM calibration\iter_1",param_new)

C:\Users\xchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide


Directory exists.
✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.46 seconds
✓ skims read. Time elapsed: 30.14 seconds
✓ trip table read. Time elapsed: 44.17 seconds
✓ zonal variable tables generated. Time elapsed: 44.65 seconds
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 44.67 seconds
✓ Trips for 0_PK calculated. Time elapsed: 63.14 seconds
✓ Trips for 1_PK calculated. Time elapsed: 80.32 seconds
✓ Trips for 0_OP calculated. Time elapsed: 97.24 seconds
✓ Trips for 1_OP calculated. Time elapsed: 113.62 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 116.22 seconds
✓ Trips for 0_PK calculated. Time elapsed: 127.82 seconds
✓ Trips for 1_PK calculated. Time elapsed: 138.84 seconds
✓ Trips for 0_OP calculated. Time elapsed: 150.01 seconds
✓ Trips for 1_OP calculated. Time elapsed: 161.00 seconds
Mode choice for NHB started.
✓ Parameter table for NHB generated. Time ela

In [16]:
for i in range(2,10):
    update_ASC(sm_calibration_run_path+ "SM calibration\iter_{}\MC_mode_share.xlsx".format(i-1), 
               sm_calibration_run_path+ "SM calibration\param_iter{}.xlsx".format(i-1),
               sm_calibration_run_path + "SM calibration\param_iter{}.xlsx".format(i))
    iter_dir = sm_calibration_run_path+ "SM calibration\iter_{}".format(i)
    new_run(iter_dir, sm_calibration_run_path + "SM calibration\param_iter{}.xlsx".format(i))

Directory exists.
✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.63 seconds
✓ skims read. Time elapsed: 28.81 seconds
✓ trip table read. Time elapsed: 41.36 seconds
✓ zonal variable tables generated. Time elapsed: 41.79 seconds
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 41.82 seconds
✓ Trips for 0_PK calculated. Time elapsed: 60.01 seconds
✓ Trips for 1_PK calculated. Time elapsed: 79.14 seconds
✓ Trips for 0_OP calculated. Time elapsed: 96.27 seconds
✓ Trips for 1_OP calculated. Time elapsed: 112.40 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 113.73 seconds
✓ Trips for 0_PK calculated. Time elapsed: 124.67 seconds
✓ Trips for 1_PK calculated. Time elapsed: 137.96 seconds
✓ Trips for 0_OP calculated. Time elapsed: 152.36 seconds
✓ Trips for 1_OP calculated. Time elapsed: 165.08 seconds
Mode choice for NHB started.
✓ Parameter table for NHB generated. Time ela

✓ Trips for 0_OP calculated. Time elapsed: 246.96 seconds
✓ Trips for 1_OP calculated. Time elapsed: 255.88 seconds
Mode choice for HBSc2 started.
✓ Parameter table for HBSc2 generated. Time elapsed: 256.66 seconds
✓ Trips for 0_PK calculated. Time elapsed: 265.34 seconds
✓ Trips for 1_PK calculated. Time elapsed: 279.09 seconds
✓ Trips for 0_OP calculated. Time elapsed: 288.34 seconds
✓ Trips for 1_OP calculated. Time elapsed: 297.27 seconds
Mode choice for HBSc3 started.
✓ Parameter table for HBSc3 generated. Time elapsed: 298.07 seconds
✓ Trips for 0_PK calculated. Time elapsed: 309.22 seconds
✓ Trips for 1_PK calculated. Time elapsed: 324.28 seconds
✓ Trips for 0_OP calculated. Time elapsed: 334.83 seconds
✓ Trips for 1_OP calculated. Time elapsed: 345.91 seconds
Directory exists.
✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.60 seconds
✓ skims read. Time elapsed: 28.67 seconds
✓ trip table read. Time elapsed: 40.55 seconds
✓ zonal variable tables generated. Ti

✓ Trips for 0_PK calculated. Time elapsed: 118.55 seconds
✓ Trips for 1_PK calculated. Time elapsed: 129.70 seconds
✓ Trips for 0_OP calculated. Time elapsed: 142.27 seconds
✓ Trips for 1_OP calculated. Time elapsed: 153.26 seconds
Mode choice for NHB started.
✓ Parameter table for NHB generated. Time elapsed: 155.07 seconds
✓ Trips for 0_PK calculated. Time elapsed: 173.30 seconds
✓ Trips for 1_PK calculated. Time elapsed: 194.28 seconds
✓ Trips for 0_OP calculated. Time elapsed: 206.78 seconds
✓ Trips for 1_OP calculated. Time elapsed: 220.06 seconds
Mode choice for HBSc1 started.
✓ Parameter table for HBSc1 generated. Time elapsed: 221.31 seconds
✓ Trips for 0_PK calculated. Time elapsed: 230.90 seconds
✓ Trips for 1_PK calculated. Time elapsed: 239.56 seconds
✓ Trips for 0_OP calculated. Time elapsed: 248.22 seconds
✓ Trips for 1_OP calculated. Time elapsed: 257.20 seconds
Mode choice for HBSc2 started.
✓ Parameter table for HBSc2 generated. Time elapsed: 257.98 seconds
✓ Trips for

In [19]:
# compare target and current model output (percentage deviation)
mc_summary = sm_calibration_run_path + "SM calibration\iter_9\MC_mode_share.xlsx"
for purpose in purposes:
    model = pd.read_excel(mc_summary,sheet_name = purpose,index_col = 0)

    modes = model.index
    df = pd.DataFrame(index = modes, columns = peak_veh)
    for pv in peak_veh:
        _t = target[(target['Purpose'] == purpose) & (target['PV'] == pv)]
        for mode in modes:
            target_share = _t[mode].values
            model_value = model.loc[mode][pv] # convert to share
            model_share = model_value / model[pv].sum()
            df.loc[mode,pv] = (abs((model_share - target_share)/target_share))[0] # percentage deviation of share
            
            
            #(model_share - target_share)[0]
    display(df.style.format('{:.2f}'))

,0_PK,1_PK,0_OP,1_OP
DA,0.02,0.01,0.05,0.01
SR2,0.03,0.03,0.05,0.02
SR3+,0.03,0.04,0.05,0.02
Walk,0.03,0.04,0.04,0.05
Bike,0.01,0.03,0.01,0.03
WAT,0.01,0.04,0.02,0.05
DAT_RT,0.02,0.02,0.03,0.01
DAT_CR,0.03,0.02,0.00,0.02
DAT_LB,0.04,0.01,0.03,0.12
DAT_B,0.01,0.03,0.01,0.05


,0_PK,1_PK,0_OP,1_OP
DA,0.00,0.01,0.00,0.01
SR2,0.00,0.01,0.00,0.01
Walk,0.00,0.04,0.00,0.04
Bike,0.03,0.02,0.03,0.03
WAT,0.01,0.04,0.02,0.03
DAT_CR,0.00,0.33,0.04,0.00
DAT_RT,0.01,0.01,0.00,0.01
DAT_LB,0.21,0.00,0.01,0.00
DAT_B,0.01,0.04,0.02,0.03
SM_RA,0.02,0.02,0.02,0.02


,0_PK,1_PK,0_OP,1_OP
DA,0.04,0.00,0.04,0.01
SR2,0.04,0.00,0.04,0.00
Walk,0.02,0.01,0.03,0.03
Bike,0.00,0.01,0.01,0.00
WAT,0.03,0.01,0.02,0.01
DAT_CR,0.04,0.00,0.02,0.03
DAT_RT,0.00,1.42,0.04,0.17
DAT_LB,0.08,0.04,0.12,0.17
DAT_B,0.01,0.04,0.03,0.03
SM_RA,0.02,0.02,0.02,0.02


,0_PK,1_PK,0_OP,1_OP
SR2,0.03,0.00,0.03,0.01
Walk,0.01,0.00,0.02,0.01
Bike,0.01,0.01,0.02,0.02
WAT,0.02,0.01,0.01,0.02
DAT_CR,0.04,0.01,0.03,0.02
DAT_RT,0.04,0.01,0.02,0.02
DAT_LB,0.03,0.29,0.04,0.01
SM_RA,0.03,0.01,0.03,0.02
SM_SH,0.03,0.10,0.03,0.02


,0_PK,1_PK,0_OP,1_OP
SR2,0.03,0.00,0.03,0.00
Walk,0.00,0.01,0.00,0.00
Bike,0.02,0.01,0.02,0.01
WAT,0.04,0.01,0.05,0.01
DAT_CR,0.02,0.04,0.03,0.03
DAT_RT,0.01,0.02,0.04,0.04
DAT_LB,0.04,0.01,0.00,0.01
SM_RA,0.03,0.02,0.02,0.01
SM_SH,0.03,0.05,0.02,0.18


,0_PK,1_PK,0_OP,1_OP
DA,0.04,0.01,0.01,0.01
SR2,0.02,0.01,0.01,0.00
Walk,0.05,0.03,0.00,0.04
Bike,0.01,0.04,0.00,0.04
WAT,0.04,0.04,0.01,0.04
DAT_CR,0.03,0.01,0.02,0.04
DAT_RT,0.02,0.01,0.01,0.01
DAT_LB,0.02,0.02,0.03,0.02
SM_RA,0.05,0.02,0.05,0.04
SM_SH,0.05,0.02,0.05,0.04


In [ ]:
# Calibration for regular modes (without SM)
target = pd.read_csv(r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Utilities\smart mobility\regular modes\MC_TargetShares.csv")
calibration_run_path = r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Utilities\smart mobility\regular modes\\"
mc_summary_file = calibration_run_path+ "base_share_for_calib.xlsx"
param_file ="X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Mode Choice\param_calib_0716.xlsx"
param_new = calibration_run_path"param_iter1.xlsx"

if 'mode_choice' in sys.modules:  
    del sys.modules["mode_choice"]

if 'mc_util' in sys.modules:  
    del sys.modules["mc_util"]

if 'config' in sys.modules:
    del sys.modules['config']

import mode_choice; import mc_util; import config

update_ASC(mc_summary_file, param_file, param_new)
new_run(calibration_run_path + "iter_1",param_new)

for i in range(2,10):
    update_ASC(calibration_run_path+ "iter_{}\MC_mode_share.xlsx".format(i-1), 
               calibration_run_path+ "param_iter{}.xlsx".format(i-1),
               calibration_run_path + "param_iter{}.xlsx".format(i))
    iter_dir = calibration_run_path+ "iter_{}".format(i)
    new_run(iter_dir, calibration_run_path + "param_iter{}.xlsx".format(i))








In [80]:
# trips by TNC
TNC_trips = pd.read_excel(r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Data\TNCs\MA-Rideshare-2017.xlsx")
# want number of trips going to/from each town by 0_veh hh and 1+ veh hh
data_path = r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Data\CTPS_Data\ModelOutputs\2040\\"
pre_MC_trip_file = data_path + "pre_MC_trip_6_purposes.omx"
taz = pd.read_csv(data_path + "..\..\TAZ_and_Town_Polygons\SW_TAZ_2010.csv")
taz = taz[taz['ID_FOR_CS']<10000]
pre_MC_trip_table = mc_util.store_omx_as_dict(pre_MC_trip_file)


In [81]:
# rename towns in TNC data for matching.
TNC_trips.replace('BARNSTABLE','BARNSTABLE TOWN',inplace =True)
TNC_trips.replace('MANCHESTER','MANCHESTER-BY-THE-SEA',inplace =True)
unused_towns = set(TNC_trips.TOWN) - set(taz.TOWN.str.split(',',expand =True)[0])
TNC_trips.drop(TNC_trips.index[TNC_trips.TOWN.isin(unused_towns)],inplace =True)

In [82]:
import re
def find_trips(row):
    town = row.TOWN
    hh_0 = list(filter(re.compile('.*_0Auto').match, list(pre_MC_trip_table.keys())))
    hh_1 = list(filter(re.compile('.*_wAuto').match, list(pre_MC_trip_table.keys())))
    origin_trips_HH0 = 0
    dest_trips_HH0 = 0
    origin_trips_HH1 = 0
    dest_trips_HH1 = 0
    for i in hh_0:
        origin_trips_HH0 += pre_MC_trip_table[i][:2730,:2730][taz['TOWN']==town+',MA',:].sum()
        dest_trips_HH0 += pre_MC_trip_table[i][:2730,:2730][:,taz['TOWN']==town+',MA'].sum()
    for i in hh_1:
        origin_trips_HH1 += pre_MC_trip_table[i][:2730,:2730][taz['TOWN']==town+',MA',:].sum()
        dest_trips_HH1 += pre_MC_trip_table[i][:2730,:2730][:,taz['TOWN']==town+',MA'].sum()       
    return pd.Series({'origin_trips_HH0':origin_trips_HH0,
    'dest_trips_HH0':dest_trips_HH0,                  
    'origin_trips_HH1':origin_trips_HH1,    
    'dest_trips_HH1':dest_trips_HH1})

In [83]:
pd.concat([TNC_trips,TNC_trips.apply(find_trips,axis = 1)],axis = 1).to_csv(r'X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Data\TNCs\trips_veh_ownership.csv')

In [65]:
# smart mobility management policy parameter file
sm_baseline_param_file = r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Scenarios\param_calib_SM.xlsx"
sm_mngpol_param_file = r"X:\Proj\2018\180021 - ISE_GHGMitAnly\001\Model\Scenarios\param_calib_SM_mngpol.xlsx"

writer = pd.ExcelWriter(sm_mngpol_param_file, engine = 'openpyxl')
for purpose in ['HBW', 'HBO', 'NHB', 'HBSc1', 'HBSc2', 'HBSc3']:
    bs_param = pd.read_excel(sm_baseline_param_file, sheet_name = purpose,index_col = 0)
    # raise SM_RA cost coefficient by 25%
    bs_param.loc['SM_RA','Cost'] *= 1.25
    bs_param.to_excel(writer, sheet_name = purpose)
writer.save()